<a href="https://colab.research.google.com/github/mahb97/Wake2vec/blob/main/Wake2vec_Llama_3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wake2Vec “*F* the Embeddings” (T4 Edn)
This notebook is a colab-friendly, embedding-only finetune pipeline for large decoder LMs (Mistral-7B / Llama-2-13B / Llama-3.1-8B) using a Wake lexicon injection. It adds Joyce-specific tokens, initializes them on a sphere, and trains only the input embedding rows (optionally with a minimal LoRA r=1 on q_proj to satisfy quantized-training rules). The goal is to bend local geometry (neighbors, isotropy) while keeping the rest of the model frozen.








# p2
max_steps: 1500-2500

lr: 5e-4 → 1e-4 (cosine decay)

batch_size: 1

grad_accum: 16

Custom loss = LM_loss + λ₁·attraction + λ₂·repulsion + λ₃·morphological + λ₄·adversarial

guardrail

In [ ]:
# NUCLEAR OPT
!pip uninstall -y torch torchvision torchaudio triton bitsandbytes transformers accelerate peft fastai timm

# rr
import os
os.kill(os.getpid(), 9)

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Found existing installation: triton 3.4.0
Uninstalling triton-3.4.0:
  Successfully uninstalled triton-3.4.0
Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: fastai 2.8.4
Uninstalling fastai-2.8.4:
  Successfully uninstalled fastai-2.8.4
Found existing

In [1]:
# Stop TorchAO
import os
os.environ["TRANSFORMERS_NO_TORCHAO"] = "1"

# compat versions
!pip install -q --no-cache-dir \
    torch==2.5.1 \
    triton==3.1.0 \
    bitsandbytes==0.43.3 \
    transformers==4.45.2 \
    accelerate==0.34.2 \
    peft==0.13.2

# Verify
import torch, bitsandbytes as bnb, triton
print("torch:", torch.__version__, "| cuda:", torch.version.cuda)
print("bnb:", bnb.__version__, "| triton:", triton.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 141.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 282.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 MB 230.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 181.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 379.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 353.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 238.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 239.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 250.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 417.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 221.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
# stop TorchAO
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from getpass import getpass
from huggingface_hub import login

HF_TOKEN = getpass("Paste your HF token (hidden): ")
login(token=HF_TOKEN, add_to_git_credential=True)

Paste your HF token (hidden): ··········


In [15]:
# Val dataset
from torch.utils.data import Dataset
import torch

class BlockDataset(Dataset):
    """Sliding window dataset for causal LM training."""
    def __init__(self, txt_path, tokenizer, seq_len=512, stride=512):
        with open(txt_path, 'r', encoding='utf-8') as f:
            text = f.read()
        self.tokens = tokenizer(text, add_special_tokens=False)['input_ids']
        self.seq_len = seq_len
        self.stride = stride
        self.starts = list(range(0, len(self.tokens) - seq_len + 1, stride))

    def __len__(self):
        return len(self.starts)

    def __getitem__(self, idx):
        start = self.starts[idx]
        chunk = self.tokens[start:start + self.seq_len]
        return {'input_ids': torch.tensor(chunk, dtype=torch.long)}

from transformers import AutoTokenizer

print("Testing dataset creation...")
tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B", use_fast=True)

ds = BlockDataset("/content/FW_TEXT.txt", tok, seq_len=512, stride=512)
print(f"  Dataset size: {len(ds)} blocks")
print(f"  First block shape: {ds[0]['input_ids'].shape}")

print("\nDataset validated")

Testing dataset creation...


Token indices sequence length is longer than the specified maximum sequence length for this model (378828 > 131072). Running this sequence through the model will result in indexing errors


  Dataset size: 739 blocks
  First block shape: torch.Size([512])

Dataset validated


In [16]:
# Helper functions
def read_lines(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]

class BlockDataset(Dataset):
    """Sliding window dataset for causal LM training."""
    def __init__(self, txt_path, tokenizer, seq_len=512, stride=512):
        with open(txt_path, 'r', encoding='utf-8') as f:
            text = f.read()
        self.tokens = tokenizer(text, add_special_tokens=False)['input_ids']
        self.seq_len = seq_len
        self.stride = stride
        self.starts = list(range(0, len(self.tokens) - seq_len + 1, stride))

    def __len__(self):
        return len(self.starts)

    def __getitem__(self, idx):
        start = self.starts[idx]
        chunk = self.tokens[start:start + self.seq_len]
        return {'input_ids': torch.tensor(chunk, dtype=torch.long)}

In [17]:
# Pre-train
import os

print("GPU Check:")
print(f"  Device: {torch.cuda.get_device_name(0)}")
print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"  Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

print("\nFile Check:")
files = {
    "Wake Lexicon": "/content/wake_lexicon.txt",
    "FW Text": "/content/FW_TEXT.txt"
}
for name, path in files.items():
    exists = os.path.exists(path)
    status = "Found" if exists else "MISSING"
    print(f"  {name}: {status}")
    if exists:
        size = os.path.getsize(path) / 1024
        print(f"    Size: {size:.1f} KB")

GPU Check:
  Device: Tesla T4
  Memory: 15.83 GB
  Allocated: 15.63 GB

File Check:
  Wake Lexicon: Found
    Size: 403.0 KB
  FW Text: Found
    Size: 1358.4 KB


In [18]:
import gc
import torch

# Verify clean slate
torch.cuda.empty_cache()
gc.collect()

print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
print(f"GPU memory cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

GPU memory allocated: 10.42 GB
GPU memory cached: 15.66 GB


In [19]:
# memory footprint
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

MODEL_NAME = "meta-llama/Llama-3.2-3B"

print(f"Testing {MODEL_NAME} load on T4")

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("\nLoading tokenizer...")
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
print(f"  Vocab size: {len(tok)}")

print("\nLoading model with 4-bit quantization...")
torch.cuda.empty_cache()
initial_mem = torch.cuda.memory_allocated(0) / 1e9

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb,
    torch_dtype=torch.float16,
    device_map="auto"
)

loaded_mem = torch.cuda.memory_allocated(0) / 1e9
print(f"  Model loaded: {loaded_mem:.2f} GB")
print(f"  Delta: {loaded_mem - initial_mem:.2f} GB")

# Validate forward pass
print("\nTesting forward pass...")
test_ids = torch.tensor([[1, 2, 3, 4, 5]], device="cuda")
with torch.no_grad():
    out = model(test_ids)
    print(f"  Output shape: {out.logits.shape}")
    print(f"  Memory after forward: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

print(f"\nPeak memory: {torch.cuda.max_memory_allocated(0) / 1e9:.2f} GB")
print("Model validated successfully")

# Cleanup
del model, tok
torch.cuda.empty_cache()
print("Memory cleared for main run")

Testing meta-llama/Llama-3.2-3B load on T4

Loading tokenizer...
  Vocab size: 128256

Loading model with 4-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  Model loaded: 9.07 GB
  Delta: -1.35 GB

Testing forward pass...
  Output shape: torch.Size([1, 5, 128256])
  Memory after forward: 9.08 GB

Peak memory: 15.63 GB
Model validated successfully
Memory cleared for main run


In [ ]:
import os, math, json, random, torch
from torch.utils.data import Dataset, DataLoader
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          TrainingArguments, Trainer, set_seed)
from peft import LoraConfig, get_peft_model

SEED=42; set_seed(SEED)
MODEL_NAME = "meta-llama/Llama-3.2-1B"
WAKE_LEX_PATH = "/content/wake_lexicon.txt"
CORPUS_TXT    = "/content/finnegans_wake.txt"
RUN_DIR       = "/content/wake_llama_embs"
SEQ_LEN=512; STRIDE=512
MAX_STEPS=1100; LOG_STEPS=20; SAVE_STEPS=200
LR=5e-5                     # ↓ safer than 8e-4 for fp16 stability, adjusted for stability
GRAD_ACCUM=8
REPULSION_W=0.0
TARGET_NORM=None
MAX_ROW_NORM=None
REPORT_SAMPLE=1500

os.makedirs(RUN_DIR, exist_ok=True)

# 4-bit load (bnb)
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                         bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16, # Changed to bfloat16 for T4 compatibility
                         llm_int8_enable_fp32_cpu_offload=True)

# pass token=HF_TOKEN if the repo is gated
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=bnb, torch_dtype=torch.bfloat16, device_map="auto", # Changed to bfloat16 for T4 compatibility
    max_memory={0: "13GB", "cpu": "30GB"}
)
model.config.use_cache = False
model.config.attn_implementation = "eager"
model.config.tie_word_embeddings = True
if hasattr(model, "tie_weights"): model.tie_weights()

# tiny PEFT adapter (kept FROZEN)
peft_cfg = LoraConfig(r=1, lora_alpha=1, lora_dropout=0.0,
                      target_modules=["q_proj"], bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, peft_cfg)
for n,p in model.named_parameters(): p.requires_grad=False

# inject wake lexicon
def read_lines(p):
    return [x.strip() for x in open(p, encoding="utf-8") if x.strip()] if os.path.exists(p) else []
wake = read_lines(WAKE_LEX_PATH)
missing = [t for t in wake if tok.convert_tokens_to_ids(t)==tok.unk_token_id]
num_added = tok.add_tokens(missing, special_tokens=False)

old_vocab = model.get_input_embeddings().weight.shape[0]
model.resize_token_embeddings(len(tok))
wte = model.get_input_embeddings()
if hasattr(model, "lm_head"): model.lm_head.weight = wte.weight

# spherical kick init
with torch.no_grad():
    base = wte.weight[:old_vocab]; dim = base.shape[1]
    std = base.std().item(); base_radius = std * math.sqrt(dim)
    target_radius = TARGET_NORM or (1.5 * base_radius)
    if num_added>0:
        new = torch.randn((num_added, dim), device=wte.weight.device)
        new = new/(new.norm(dim=1, keepdim=True)+1e-8)*target_radius
        wte.weight.data[old_vocab:old_vocab+num_added] = new

# trainables = ONLY embeds
wte.weight.requires_grad=True
new_rows = torch.arange(old_vocab, old_vocab+num_added, device=wte.weight.device) if num_added>0 else None
base_rows = torch.arange(0, old_vocab, device=wte.weight.device)
def mask_grad(grad):
    if grad is None or new_rows is None: return grad
    grad[base_rows]=0; return grad
wte.weight.register_hook(mask_grad)

def clamp_rows_(emb, max_norm):
    if max_norm is None or new_rows is None: return
    rows = emb.weight.data[old_vocab:old_vocab+num_added]
    norms = rows.norm(dim=1, keepdim=True).clamp_min(1e-8)
    scale = (max_norm/norms).clamp_max(1.0)
    emb.weight.data[old_vocab:old_vocab+num_added] = rows*scale

# robust torch-only dataset
class BlockDataset(Dataset):
    def __init__(self, path, tokenizer, seq_len=512, stride=512):
        if not os.path.exists(path):
            stub = ("riverrun, past Eve and Adam’s, from swerve of shore to bend of bay, "
                    "brings us by a commodius vicus of recirculation to Howth Castle and Environs. ")*2000 # Increased stub length for more data if path is empty
            text = stub
        else:
            text = open(path, "r", encoding="utf-8").read()
        ids = tokenizer(text, add_special_tokens=False)["input_ids"]
        blocks=[]
        # More robust chunking logic:
        for i in range(0, max(1, len(ids)-seq_len), stride):
            chunk = ids[i:i+seq_len]
            if len(chunk) >= seq_len//2: # Only append if chunk is at least half the seq_len
                blocks.append(chunk[:seq_len])
        self.blocks = blocks
    def __len__(self): return len(self.blocks)
    def __getitem__(self, idx):
        ids = torch.tensor(self.blocks[idx], dtype=torch.long)
        return {"input_ids": ids, "labels": ids.clone(), "attention_mask": torch.ones_like(ids)}

train_ds = BlockDataset(CORPUS_TXT, tok, SEQ_LEN, STRIDE)
print(f"[Data] chunks={len(train_ds)}; tokens/step={SEQ_LEN}")

# tiny sanity
sample = train_ds[0]
assert sample["input_ids"].ndim==1 and sample["labels"].ndim==1
assert sample["input_ids"].shape[0]==SEQ_LEN and sample["labels"].shape[0]==SEQ_LEN
assert sample["input_ids"].sum().item()!=0, "Input looks empty; check CORPUS_TXT or tokenizer."

# geometry receipts (pre)
@torch.no_grad()
def isotropy(M):
    u, s, v = torch.pca_lowrank(M - M.mean(0, keepdim=True), q=min(128, M.shape[1]-1))
    return float((s.max() / s.min().clamp_min(1e-8)))

@torch.no_grad()
def pip_loss(A, B):
    return float(torch.norm((A@A.T)-(B@B.T), p='fro')/(A.shape[0]**2))

@torch.no_grad()
def topk_overlap(M1, M2, k=10, sample=1000):
    W1 = M1/(M1.norm(dim=1, keepdim=True)+1e-8); W2 = M2/(M2.norm(dim=1, keepdim=True)+1e-8)
    vocab = W1.shape[0]; idxs = random.sample(range(vocab), min(sample, vocab))
    acc = 0.0
    for i in idxs:
        c1 = torch.topk(W1 @ W1[i], k+1).indices.tolist(); c1=[j for j in c1 if j!=i][:k]
        c2 = torch.topk(W2 @ W2[i], k+1).indices.tolist(); c2=[j for j in c2 if j!=i][:k]
        acc += len(set(c1)&set(c2))/k
    return float(acc/len(idxs))

with torch.no_grad():
    pre_full = wte.weight.detach().clone().cpu()
    pre_new  = pre_full[old_vocab:old_vocab+num_added].clone() if num_added>0 else torch.empty(0, pre_full.shape[1])

# custom Trainer to
#(a) only optimize embeddings,
#(b) add repulsion,
#(c) clamp,
#(d) clip grads
from transformers import Trainer
class EmbOnlyTrainer(Trainer):
    def create_optimizer(self):
        from torch.optim import AdamW
        if not hasattr(self, "optimizer") or self.optimizer is None:
            self.optimizer = AdamW([{"params": [wte.weight], "lr": LR, "weight_decay": 0.0}],
                                   betas=(0.9, 0.999), eps=1e-8)
        return self.optimizer
    def compute_loss(self, model, inputs, return_outputs=False):
        out = model(**inputs, use_cache=False)
        loss = out.loss
        if torch.isnan(loss) or torch.isinf(loss):
            raise RuntimeError("NaN/Inf loss detected — check data/labels.")
        if num_added and num_added>1 and REPULSION_W>0:
            E = model.get_input_embeddings().weight[old_vocab:old_vocab+num_added]
            E = E - E.mean(0, keepdim=True); E = E/(E.norm(dim=1, keepdim=True)+1e-8)
            sims = (E @ E.t()); repul = (sims - torch.eye(E.shape[0], device=E.device)).pow(2).mean()
            loss = loss + REPULSION_W*repul
        return (loss, out) if return_outputs else loss
    def training_step(self, *args, **kwargs):
        out = super().training_step(*args, **kwargs)
        clamp_rows_(model.get_input_embeddings(), MAX_ROW_NORM)
        return out

steps_per_epoch = max(1, len(train_ds) // GRAD_ACCUM)
print(f"[Epochs] steps/epoch≈{steps_per_epoch} | epochs@{MAX_STEPS}≈{MAX_STEPS/steps_per_epoch:.2f}")

args = TrainingArguments(
    output_dir=RUN_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    max_steps=MAX_STEPS,
    warmup_steps=max(20, MAX_STEPS//20),
    lr_scheduler_type="cosine",
    weight_decay=0.0,
    fp16=False, bf16=True, # Aligning with bfloat16 compute dtype for T4
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS, save_total_limit=6,
    evaluation_strategy="no",
    report_to="none",
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
)

trainer = EmbOnlyTrainer(model=model, args=args, train_dataset=train_ds, data_collator=None)
print(f"[Run] base={MODEL_NAME} | steps={MAX_STEPS} | seq_len={SEQ_LEN} | accum={GRAD_ACCUM} | LR={LR}")
trainer.train()

# save artifacts
save_dir = os.path.join(RUN_DIR, "embedding_only"); os.makedirs(save_dir, exist_ok=True)
torch.save(wte.weight.detach().cpu(), os.path.join(save_dir, "embed_tokens.pt"))
with open(os.path.join(save_dir, "added_tokens.json"), "w") as f:
    json.dump({"added_tokens": missing, "old_vocab": old_vocab, "num_added": num_added}, f, indent=2)
tok.save_pretrained(RUN_DIR)

# receipts (post)
with torch.no_grad():
    post_full = wte.weight.detach().clone().cpu()
    post_new  = post_full[old_vocab:old_vocab+num_added].clone() if num_added>0 else torch.empty(0, pre_full.shape[1])

report = {
  "model": MODEL_NAME, "added_tokens": int(num_added), "old_vocab": int(old_vocab),
  "pip_loss_full": pip_loss(pre_full, post_full),
  "topk_overlap_all": topk_overlap(pre_full, post_full, k=10, sample=min(REPORT_SAMPLE, pre_full.shape[0]-1)),
  "isotropy_pre": isotropy(pre_full), "isotropy_post": isotropy(post_full),
  "pip_loss_new_rows": (pip_loss(pre_new, post_new) if num_added>1 else None),
  "isotropy_new_rows": (isotropy(post_new) if num_added>1 else None),
}
json.dump(report, open(os.path.join(RUN_DIR, "geometry_report.json"), "w"), indent=2)
print("\n=== GEOMETRY REPORT ===")
for k,v in report.items(): print(f"{k}: {v}")

[Data] chunks=167; tokens/step=512


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


[Epochs] steps/epoch≈20 | epochs@1100≈55.00
[Run] base=meta-llama/Llama-3.2-1B | steps=1100 | seq_len=512 | accum=8 | LR=5e-05


Step,Training Loss
20,3.148400
